# 1 Zero-shot Learning

## DAP模型

《Learning To Detect Unseen Object Classes by Between-Class Attribute Transfer》

典型的DAP模型具有三层结构，第一层为原始输入层，第二层为P维特征空间，第三层是输出层，输出模型对样本类别的判断

## ALE模型

《Label-Embedding for Attribute-Based Classification》

## SAE模型

《Semantic Autoencoder for Zero-Shot learning》

## SCoRE

《Semantically Consistent Regularization for Zero-Shot Recognition》

## 全局属性

In [2]:
import os
path_root = r'D:\softfiles\workspace\games\zhi_jiang\original_data'
path_train = os.path.join(path_root, 'DatasetA_train_20180813')
save_dir = r'D:\softfiles\workspace\games\zhi_jiang\model'
path_train_with_numpy = r'D:\softfiles\workspace\games\zhi_jiang\preprocess_data'
model_dir = r'D:\softfiles\workspace\games\zhi_jiang\model'

## model

In [3]:
import os
from keras.layers import Dense, Flatten, Embedding, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.applications import vgg19
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

Using TensorFlow backend.


In [3]:
base_model = vgg19.VGG19(include_top=False, weights=None, input_shape=(64, 64, 3))

# FNet
x = base_model.output
x = Flatten()(x)
x = Dense(2048, activation='sigmoid', name='fc6')(x)
x = Dense(2048, activation='sigmoid', name='fc7')(x)

# Fnet
model_fnet = Model(inputs=base_model.input, outputs=x)
# model_fnet.summary()

# ENet
x = model_fnet.output
x = Dense(30, activation='sigmoid', name='enet')(x)
model_enet = Model(inputs=model_fnet.input, outputs=x)
# model_enet.summary()

model_enet.compile(optimizer='adam', loss='mse', metrics=['acc'])

In [ ]:
train_x = np.load(os.path.join(path_train_with_numpy, 'train_x.npy'))
train_x = train_x / 225
train_y = np.load(os.path.join(os.path.join(path_train_with_numpy, 'train_y.npy')))
                  
model_enet.fit(train_x, train_y,
               batch_size=128,
               epochs=10,
               validation_split=0.2,
               shuffle=True)

Train on 30576 samples, validate on 7645 samples
Epoch 1/10
18688/30576 [=================>............] - ETA: 3:20 - loss: 0.0911 - acc: 0.0490

In [233]:
Model.fit?

### 数据处理

In [10]:
# 把所有图片保存成（，64， 64， 3）
from PIL import Image
# temp_list = ['0080ff711d1aff18b8f46ee25f686bbd.jpeg', 'd1f11b84f776416bd87b93ee6b2f7698.jpeg']

path_train_image = os.path.join(path_train, 'train')

train_x_np = np.zeros((len(train_x_n), 64, 64, 3), dtype='float32')
val_x_np = np.zeros((len(val_x_n), 64, 64, 3), dtype='float32')

for i, fpath in enumerate(train_x_n):
    with Image.open(os.path.join(path_train_image, fpath)) as f:
        if f.mode != 'RGB':
            f = f.convert('RGB')
        train_x_np[i] = np.asarray(f, dtype='float32')
        
for i, fpath in enumerate(val_x_n):
    with Image.open(os.path.join(path_train_image, fpath)) as f:
        if f.mode != 'RGB':
            f = f.convert('RGB')
        val_x_np[i] = np.asarray(f, dtype='float32')

# train_y
train_y_np = np.zeros((len(train_y_n), 30), dtype='float32')
val_y_np = np.zeros((len(val_y_n), 30), dtype='float32')

for i, label in enumerate(train_y_n):
    train_y_np[i] = np.asarray(attribute_per_class.loc[label])
    
for i, label in enumerate(val_y_n):
    val_y_np[i] = np.asarray(attribute_per_class.loc[label])
    
# 把train_x, train_y 保存成npy格式
np.save(os.path.join(path_train_with_numpy, 'train_x.npy'), train_x_np)
np.save(os.path.join(path_train_with_numpy, 'train_y.npy'), train_y_np)
np.save(os.path.join(path_train_with_numpy, 'val_x.npy'), val_x_np)
np.save(os.path.join(path_train_with_numpy, 'val_y.npy'), val_y_np)

In [8]:
from sklearn.model_selection import train_test_split
train_x_n, val_x_n, train_y_n, val_y_n = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [13]:
len(val_x_n)

7645

## 类属性与类的映像model

In [ ]:
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

att_input = Input(shape=(30,))
att_x = Dense(128, activation='sigmoid', name='att_fc1')(att_input)
att_x = Dense(256, activation='sigmoid', name='att_fc2')(att_x)
att_pred = Dense(230, activation='softmax', name='att_output')(att_x)

att_model = Model(inputs=att_input, outputs=att_pred)
att_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

att_model_name = 'zsc_att_model.{epoch:03d}.h5'
att_model_filepath = os.path.join(save_dir, att_model_name)
att_checkpoint = ModelCheckpoint(filepath=att_model_filepath,
                                 monitor='acc',
                                 verbose=1,
                                 save_best_only=True)
att_callbacks = [att_checkpoint]

att_train_x = np.asarray(attribute_per_class.loc[:,:]).astype('float32')
att_train_y_label = np.asarray(attribute_per_class.index)
att_train_y = to_categorical(range(len(att_train_y_label)))

att_model.fit(att_train_x, att_train_y, epochs=2000)

Epoch 1/2000
230/230 [==============================] - 3s 11ms/step - loss: 5.7649 - acc: 0.0000e+00
Epoch 2/2000
230/230 [==============================] - 0s 418us/step - loss: 5.5580 - acc: 0.0043
Epoch 3/2000
230/230 [==============================] - 0s 356us/step - loss: 5.5005 - acc: 0.0043
Epoch 4/2000
230/230 [==============================] - 0s 408us/step - loss: 5.4842 - acc: 0.0043
Epoch 5/2000
230/230 [==============================] - 0s 360us/step - loss: 5.4811 - acc: 0.0043
Epoch 6/2000
230/230 [==============================] - 0s 459us/step - loss: 5.4795 - acc: 0.0043
Epoch 7/2000
230/230 [==============================] - 0s 487us/step - loss: 5.4819 - acc: 0.0043
Epoch 8/2000
230/230 [==============================] - 0s 418us/step - loss: 5.4772 - acc: 0.0043
Epoch 9/2000
230/230 [==============================] - 0s 462us/step - loss: 5.4761 - acc: 0.0043
Epoch 10/2000
230/230 [==============================] - 0s 513us/step - loss: 5.4747 - acc: 0.0087
Epoch 

In [227]:
att_model.save(os.path.join(model_dir, 'att_model.h5'))

In [162]:
# att_model测试
temp_test = np.asarray(attribute_per_class.loc['ZJL240']).astype('float32')
temp_test = np.expand_dims(temp_test, 0)
print(temp_test)
temp_test[0,6] = 0.6
print(temp_test)
args = np.argmax(att_model.predict(temp_test))
att_train_y_label[args]

[[1.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.5 0.  0.  0.  0.  0.  1.  0.  0.
  0.5 0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0. ]]
[[1.  0.  0.  0.  0.  0.  0.6 0.5 0.  0.5 0.  0.  0.  0.  0.  1.  0.  0.
  0.5 0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0. ]]


'ZJL240'

## 提取label_list
label_list = {'ZJL1': 'goldfish','ZJL10': 'tarantula','ZJL100': 'drumstick'...}

In [4]:
label_list = {}
with open(os.path.join(path_train, 'label_list.txt'), 'rb') as f:
    for line in f.readlines():
        label_id, label_name = line.strip().decode().split()
        label_list[label_id] = label_name
print('共有%s个类别' % len(label_list))

共有230个类别


## 提取attribute list
```python
attribute_list = ['is animal','is transportation','is clothes'...]
```

In [5]:
attribute_list = []
with open(os.path.join(path_train, 'attribute_list.txt'), 'rb') as f:
    for line in f.readlines():
        _, attribute_name = line.strip().decode().split('\t')
        attribute_list.append(attribute_name)
print('每个类别共有%s个属性' % len(attribute_list))

每个类别共有30个属性


## 提取每个类别对应的属性编码

In [6]:
import pandas as pd
temp_head = attribute_list.copy()
temp_head.insert(0, 'label')
attribute_per_class = pd.read_csv(os.path.join(path_train, 'attributes_per_class.txt'),
                                  sep='\t', names=temp_head, index_col=0)

## 提取训练图片路径和label

In [7]:
train_x, train_y = [], []
with open(os.path.join(path_train, 'train.txt'), 'rb') as f:
    for line in f.readlines():
        temp_path, temp_label = line.strip().decode().split('\t')
        train_x.append(temp_path)
        train_y.append(temp_label)
print('共训练%s张图片。' % len(train_x))
print('训练数据有%s个类别。' % len(set(train_y)))

共训练38221张图片。
训练数据有190个类别。
